In [1]:
import pandas as pd
import numpy as np
import time
import statsmodels.api as sm
from tqdm import tqdm

In [2]:
#direct = ('E:\\Dropbox\\A2Teaching\\COMM486_2018\\week7\\Investment2018\\')
#direct = 'C:\\Users\\adlaif\\Dropbox\A2Teaching\BAFI519_2019\class9\Investment2018\\'
direct = 'PortStrat2020/'

#%%############################################################################
# Step 1: Preparing the CRSP file
###############################################################################
timer = time.time() # record the current time, so we can measure how long the code takes to run

# load data
crsp = pd.read_csv(direct+'CRSP_Monthly_2018.csv')

## formatting
# You should see that one of the important variables 'RET' (return) is not a number but 'object'.
# It is preferable to have this variable as a number, which Python denotes as float64 (float64 is just a special way of saying that a variable is a number)
# If you are interested search for 'floating point number'on internet. But it is computer-science issue!

# Changes the returns to number format
crsp['RET'] = pd.to_numeric(crsp['RET'],errors='coerce') 

# Change the dateformat
crsp['date'] = pd.to_datetime(crsp['date'], format='%Y%m%d')

# create monthly date variable
crsp['datem'] = crsp['date'].dt.to_period('M')

## Some basic data cleaning
# keep only common shares
crsp = crsp[(crsp.SHRCD==10) | (crsp.SHRCD==11)] 

# keep only stocks from NYSE, AMEX and NASDAQ
crsp = crsp[(crsp.EXCHCD==1) | (crsp.EXCHCD==2) | (crsp.EXCHCD==3)] 

# excludes prices below $1 
#crsp = crsp[crsp.PRC >= 1]

# make sure that there are no duplicates
crsp = crsp.drop_duplicates(subset=['date','PERMNO'])

# get crsp returns in "wide" format (i.e. "pivot" the crsp data set)
ret = crsp.pivot(index='datem',columns='PERMNO',values='RET')

print('Step 1 completed in %.1fs' % (time.time()-timer)) # show how long it took to run this code block


Step 1 completed in 13.4s


In [6]:
#%%############################################################################
# Step 2b: Calculate Residuals
###############################################################################

# this part takes a while to run (20min on Julian's laptop)
# set calculateBetas_m=True to run this section, calculateBetas_m=False to skip
calculateResid_m = True

if calculateResid_m:
    timer = time.time() # record the current time, so we can measure how long the code takes to run
    
    ## prepare data
    # for performance reasons, we mostly work with numpy arrays instead of pandas in this section
    # load fama french factors
    ff = pd.read_csv(direct+'F-F_Research_Data_Factors_2018.csv')
    #factors = ff.copy()
    #print(ff)
    # align the dates of the factors with the returns
    ff['DATE'] = pd.to_datetime(ff['DATE'],format='%Y%m').dt.to_period('M')
    ff.set_index('DATE',inplace=True)
    
    ff = ff.reindex(ret.index)
    
    # divide by 100 (FF data is stored in percent)
    ff = ff / 100
    #print(ff)
    # get market factor and excess returns as numpy arrays
    #mkt = ff['Mkt-RF'].values
    factors = ff[['Mkt-RF', 'SMB', 'HML']].values
    eret = ret.sub(ff['RF'].values,axis=0).values # subtract ff['RF'] from each column of ret
    
    
    ## calculate betas
    # nmonths: calculate betas based on the most recent nmonths number of months
    nmonths = 3*12
    
    # minobs: require at least this number of non-nan observations to calculate beta, otherwise beta is nan
    minobs = 3*12
    
    # calculate
    #beta = np.empty(eret.shape)
    rmom12_2 = np.empty(eret.shape)
    rmom12_2[:,:] = np.NAN
    for t in tqdm(range(nmonths-1,eret.shape[0])):
        for j in range(0,eret.shape[1]):
            eret_subset = eret[t-nmonths+1:t+1,j]
            mask = ~np.isnan(eret_subset) # observations in range with non-nan values
            if(sum(mask) >= minobs):
                # from the regression, only obtain residuals from t-2 to t-12 
                resid12_2 = sm.OLS(eret_subset[mask],
                                    sm.add_constant(factors[t-nmonths+1:t+1][mask])).fit().resid[24:35]
                # calculate residual momentum
                rmom12_2[t,j] = resid12_2.sum() / resid12_2.std()
                
    
    # convert results to pandas dataframe
    df_rmom12_2 = pd.DataFrame(rmom12_2, index=ret.index, columns=ret.columns)
    del rmom12_2
    
    print('Step 2b completed in %.1fs' % (time.time()-timer)) # show how long it took to run this code block

100%|██████████| 421/421 [49:15<00:00,  7.02s/it]

Step 2b completed in 2955.4s


In [8]:
df_rmom12_2.to_csv('rmom12_2_1980_2017.csv')